# Repairs of Python.NET 3 breaking changes to internal tests

## 0 Load a runtime before calling `import clr`


In order to access .NET assemblies (`.dll` files), one must load an available runtime before executing the
`import clr` statement. (If one calls `import clr` before specifying a runtime, Python.NET will load a default
runtime which may **not** be compatible with the installed Orchid assemblies.

To make this easier, when we `import` the `orchid` package, the Orchid Python API will load the runtime
corresponding to the configured Orchid installation.

In [ ]:
import orchid

from orchid import (
    net_fracture_diagnostics_factory as net_factory,
)

In [ ]:
#noinspection PyPackageRequirements
import clr

In [ ]:
# noinspection PyUnresolvedReferences,PyPackageRequirements
from System import (
    ArgumentException,
    Convert,
    DateTime,
    DateTimeKind,
    DateTimeOffset,
    Int32,
    InvalidCastException,
    TimeSpan,
)

object_factory = net_factory.create()

## 1 Reduce the implicit conversions between Python types and .NET Types

### 1.1 Equality between Python `int` values and `DateTimeOffset.MaxValue` no longer supported

Python.NET 2.5.2 allowed a developer to test for equality between Python `int` values and values of type `DateTimeOffset`
by performing an implicit conversion of the `DateTimeOffset` type.

In [ ]:
try:
    print('Trying `108 == DateTimeOffset.Value`')
    108 == DateTimeOffset.MaxValue
except TypeError as nie:
    print(f'TypeError: {nie}')

In [ ]:
try:
    print('Trying `108 == TimeSpan.MaxValue`')
    108 == TimeSpan.MaxValue
except TypeError as nie:
    print(f'TypeError: {nie}')

We filed an issue with the Python.NET team. They responded with the following:

> Yes, we tried to limit the "implicit" conversions to a minimum. I don't even know, which change in particular is responsible for the behavioural ~change~ fix that you are observing here, but you are only able to compare things to a .NET object that are directly convertible to it. If you'd really require this for `DateTimeOffset` and `TimeSpan`, you could make them convertible via a [Codec](https://pythonnet.github.io/pythonnet/codecs.html). Otherwise, I'd suggest you just generate the respective comparison values using `.FromTicks`.

Python.NET 3 requires a developer to explicitly use the `DateTimeOffset.Ticks` for in the equality test.

In [ ]:
108 == DateTimeOffset.MaxValue.Ticks

In [ ]:
108 == TimeSpan.MaxValue.Ticks

In [ ]:
108 == TimeSpan.MinValue.Ticks

### 1.2 Less effort to make Python constructors "just work"

Python.NET 2.5.2 allowed expressions like `TimeSpan()`. (Note that the .NET `TimeSpan` class **does not** have a 
default constructor.) This expression is no longer supported. Instead, one must supply an argument 
(perhaps zero (0)) to the constructor, or one must use methods like `TimeSpan.FromTicks()`.

In [ ]:
try:
    print('Trying expression, `TimeSpan()`')
    TimeSpan()
except TypeError as te:
    print(f'TypeError: {te}')

In [ ]:
TimeSpan(8801).ToString()

In [ ]:
TimeSpan(0).ToString()

In [ ]:
TimeSpan.FromTicks(0).ToString()

## 2 Adding attributes with integer values requires explicit conversion

(This issue occurred in **both** internal testing and low-level script testing and so is duplicated.)

During integration testing, we discovered an issue setting an attribute with type, `Int32`, using a Python `int` value of 7. The run-time reported that the types, `Int32` and `PyInt` were incompatible.

This scenario requires significant set up.

In [ ]:
# Find the well named 'Demo_1H'
bakken = orchid.load_project('c:/src/Orchid.IntegrationTestData/frankNstein_Bakken_UTM13_FEET.ifrac')
candidate_wells = list(bakken.wells().find_by_name('Demo_1H'))
assert len(candidate_wells) == 1
demo_1h = candidate_wells[0]

In [ ]:
# Create an attribute with name, 'My New Attribute', and type, `System.Int32`
attribute_to_add_type = Int32
attribute_to_add = object_factory.CreateAttribute[attribute_to_add_type]('My New Attribute', -1)

In [ ]:
# Add newly created attribute to well, 'Demo_1H'
with orchid.dot_net_disposable.disposable(demo_1h.dom_object.ToMutable()) as mutable_well:
    mutable_well.AddStageAttribute(attribute_to_add)

In [ ]:
# Find stage number 7 in well, 'Demo_1H'
maybe_stage = demo_1h.stages().find_by_display_stage_number(7)
assert maybe_stage is not None
stage_7 = maybe_stage

In [ ]:
# Add attribute with value, 17, to stage 7, with Python `int` type.
with (orchid.dot_net_disposable.disposable(stage_7.dom_object.ToMutable())) as mutable_stage:
    # This action will fail because the attribute type is `System.Int32`
    # and `pythonnet-3.0.0.post1` **does not** implicitly equate these two types.
    try:
        mutable_stage.SetAttribute(attribute_to_add, int)
    except ArgumentException as ae:
        print(f'ArgumentException: {ae}')

In [ ]:
# Add attribute to stage 7 with a value of 17 **explicitly** converted to an `Int32`
with (orchid.dot_net_disposable.disposable(stage_7.dom_object.ToMutable())) as mutable_stage:
    mutable_stage.SetAttribute(attribute_to_add, attribute_to_add_type(7))

In [ ]:
# Verify added attribute value
ignored_object = object()
is_attribute_present, actual_attribute_value = stage_7.dom_object.TryGetAttributeValue(attribute_to_add,
                                                                                       ignored_object)
assert is_attribute_present
assert type(actual_attribute_value) == int
assert actual_attribute_value == 7

## 3 Disabled implicit conversion from C# Enums to Python `int` and back

### 3.1 Reduced need for and changed behavior of `Overloads` (`__overloads__` in Python.NET 3)

The .NET `DateTime` class has many overloaded constructors. Because version Python.NET 2.5.2 converted members 
of .NET Enum types into Python `int` values, the method resolution process could not distinguish between the
`DateTime` constructor taking 7 `System.Int32` arguments (the last specifying milliseconds) and the constructor
accepting 6 `System.Int32` values and a `DateTimeKind` member. Consequently, a developer of the Orchid Python API
had to specify an overload in order to invoke the appropriate constructor.

Under Python 2.5.2, one used the `Overloads` attribute to select a specific overload. Additionally, the
`Overloads` attribute could be queried to return a `list` of available overloads.

Under Python 3, querying the `__overloads__` (preferred but `Overloads` is also available) produces an unexpected
result.

In [ ]:
DateTime.Overloads, DateTime.__overloads__

Our working hypothesis is that the Python.NET method resolution algorithm could find any overloads for the
constructor and, therefore, produces this behavior.

Additionally, we did create a issue with the Python.NET team. In the response to our issue, the Python.NET team
indicated that `__overloads__` was not an attribute but a [property](https://realpython.com/python-property/).

In [ ]:
type(DateTime.__overloads__)

### 3.2 .NET Enum members are no longer converted to Python `int` values

Python.NET 2.5.2 implicitly converted .NET `Enum` members to Python `int` values. Python.NET 3 exposes
the (derived) .NET `Enum` type to Python.

In [ ]:
type(DateTimeKind.Utc)

In [ ]:
dir(DateTimeKind.Utc)

In [ ]:
DateTimeKind.Utc.GetType()

In [ ]:
dir(DateTimeKind.Utc.GetType())

In [ ]:
DateTimeKind.Utc.GetType().BaseType

In [ ]:
DateTimeKind.Utc.GetType().BaseType.FullName

Because Python.NET 3 retains the .NET `Enum` member, Python.NET can then resolve the `DateTime`
7-argument constructor with the `DateTimeKind` last argument without "help".

In [ ]:
DateTime(2021, 12, 1, 12, 15, 37, DateTimeKind.Utc).ToString('o')

### 3.3 Eliminated need to inherit from Python `enum.IntEnum` for compatibility with .NET Enum types

Version 2.5.2 of `pythonnet` converted values of type .NET Enum to Python `int` values. Consequently, to support 
easy comparisons between the .NET type, `Orchid.FractureDiagnostics.FormationConnectionType` and the Python 
enumeration, `native_stage_adapter.ConnectionType`, we defined `native_stage_adapter.ConnectionType` to inherit 
from `enum.IntEnum`.  This base class is not needed in `pythonnet-3.0.0.post1` because the enumeration member 
`native_stage_adapter.ConnectionType.PLUG_AND_PERF`, defined to have a value of
`Orchid.FractureDiagnostics.FormationConnectionType` is no longer of type `int` but is actually of type, 
`Orchid.FractureDiagnostics.FormationConnectionType`.

In [ ]:
# Returned `True` in `pythonnet-2.5.2`
orchid.net_date_time.TimePointTimeZoneKind.UTC == 0

In [ ]:
orchid.net_date_time.DateTimeKind

In [ ]:
orchid.net_date_time.TimePointTimeZoneKind

In [ ]:
orchid.net_date_time.TimePointTimeZoneKind.UTC

In [ ]:
orchid.net_date_time.TimePointTimeZoneKind.UTC.value

In [ ]:
# Similarly, this expression returned `True` in `pythonnet-2.5.2`
orchid.native_stage_adapter.ConnectionType == 0

In [ ]:
orchid.native_stage_adapter.FormationConnectionType

In [ ]:
orchid.native_stage_adapter.ConnectionType

In [ ]:
orchid.native_stage_adapter.ConnectionType.PLUG_AND_PERF

In [ ]:
orchid.native_stage_adapter.ConnectionType.OPEN_HOLE

In [ ]:
orchid.native_stage_adapter.ConnectionType.PLUG_AND_PERF.value.ToString()

## 4 Return values from .NET methods that return an interface are now automatically wrapped in that interface

Under `pythonnet-2.5.2`, running the following `doctest` passes:

```
    >>> start = pendulum.parse('2022-02-23T15:53:23Z')
    >>> stop = pendulum.parse('2022-02-24T05:54:11Z')
    >>> net_start = ndt.as_net_date_time(start)
    >>> net_stop = ndt.as_net_date_time(stop)
    >>> factory = create()
    >>> date_time_offset_range = factory.CreateDateTimeOffsetRange(net_start, net_stop)
    >>> (date_time_offset_range.Start.ToString('o'), date_time_offset_range.Stop.ToString('o'))
    ('2022-02-23T15:53:23.0000000+00:00', '2022-02-24T05:54:11.0000000+00:00')
```

When running the same `doctest` using `pythonnet-3.0.0.post1`, this code
encounters an unhandled exception:

```
Error
**********************************************************************
File "C:\src\orchid-python-api\orchid\net_fracture_diagnostics_factory.py", line ?, in net_fracture_diagnostics_factory.create
Failed example:
    (date_time_offset_range.Start.ToString('o'), date_time_offset_range.Stop.ToString('o'))
Exception raised:
    Traceback (most recent call last):
      File "C:/Users/larry.jones/AppData/Local/JetBrains/Toolbox/apps/PyCharm-P/ch-0/222.4459.20/plugins/python/helpers/pycharm/docrunner.py", line 138, in __run
        exec(compile(example.source, filename, "single",
      File "<doctest net_fracture_diagnostics_factory.create[6]>", line 1, in <module>
        (date_time_offset_range.Start.ToString('o'), date_time_offset_range.Stop.ToString('o'))
    TypeError: No method matches given arguments for Object.ToString: (<class 'str'>)
```

In [ ]:
import pendulum

In [ ]:
start = pendulum.parse('2022-02-23T15:53:23Z')
stop = pendulum.parse('2022-02-24T05:54:11Z')
net_start = orchid.net_date_time.as_net_date_time(start)
net_stop = orchid.net_date_time.as_net_date_time(stop)
factory = orchid.net_fracture_diagnostics_factory.create()

In [ ]:
date_time_offset_range = factory.CreateDateTimeOffsetRange(net_start, net_stop)
type(date_time_offset_range)

In [ ]:
dir(date_time_offset_range)

In [ ]:
str(date_time_offset_range)

In [ ]:
date_time_offset_range.Start.GetType().FullName

In [ ]:
net_range_start = date_time_offset_range.Start
type(net_range_start)

In [ ]:
try:
    net_range_start.ToString('o')
except TypeError as te:
    print(f'TypeError: {te}')

In [ ]:
net_range_start.ToString()

In [ ]:
type(net_range_start)

In [ ]:
net_range_start.GetType().FullName

In [ ]:
dir(net_range_start.GetType())

In [ ]:
[iface.FullName for iface in net_range_start.GetType().GetInterfaces()]

Unfortunately, .NET does not support conversion from the interface to the actual class.

In [ ]:
try:
    error_net_range_start = Convert.ChangeType(date_time_offset_range, DateTimeOffset)
    type(error_net_range_start)
except InvalidCastException as ice:
    print(f'InvalidCastException {ice}')

Although I could work around the issue by calling `ToString()`, I was not very satisfied. After posting [an issue](https://github.com/pythonnet/pythonnet/issues/2034), I received a [response](https://github.com/pythonnet/pythonnet/issues/2034#issuecomment-1332728831) that stated:

> You can access `__implementation__` (codecs applied) or `__raw_implementation__` (codecs not applied).

In [ ]:
net_range_start.__implementation__

In [ ]:
net_range_start.__raw_implementation__

In [ ]:
[o.__implementation__.ToString('o') for o in (date_time_offset_range.Start, date_time_offset_range.Stop)]